In [1]:
import requests
import json
import pandas as pd
from collections import OrderedDict
import folium
import schedule
import time

# Constants

In [2]:
ICON_MAPPING = {
    0: 'Unknown',
    1: 'Accident',
    2: 'Fog',
    3: 'Dangerous Conditions',
    4: 'Rain',
    5: 'Ice',
    6: 'Jam',
    7: 'Lane Closed',
    8: 'Road Closed',
    9: 'Road Works',
    10: 'Wind',
    11: 'Flooding',
    14: 'Broken Down Vehicle'
}

In [3]:
ICON_STYLES = {
    'Accident': {'icon': 'car', 'color': 'red'},
    'Jam': {'icon': 'road', 'color': 'orange'},
    'Road Works': {'icon': 'wrench', 'color': 'blue'},
    'Fog': {'icon': 'cloud', 'color': 'gray'},
    'Rain': {'icon': 'cloud-rain', 'color': 'blue'},
    'Ice': {'icon': 'snowflake', 'color': 'lightblue'},
    'Wind': {'icon': 'wind', 'color': 'lightgreen'},
    'Flooding': {'icon': 'tint', 'color': 'blue'},
    'Broken Down Vehicle': {'icon': 'truck', 'color': 'black'},
    'Lane Closed': {'icon': 'minus-circle', 'color': 'purple'},
    'Dangerous Conditions': {'icon': 'exclamation-triangle', 'color': 'yellow'},
    'Road Closed': {'icon': 'times-circle', 'color': 'darkred'},
    'Unknown': {'icon': 'question-circle', 'color': 'gray'}
}

# 1.

### With default fields

In [4]:
def get_traffic_incidents(api_key, bbox, 
                         baseURL = "api.tomtom.com", 
                         versionNumber = 5):
    """
    for real-time data on accidents, congestion, road closures, etc.
    """
    # [GUIDE] https://developer.tomtom.com/traffic-api/documentation/traffic-incidents/incident-details
    url = f"https://{baseURL}/traffic/services/{versionNumber}/incidentDetails?key={api_key}&bbox={bbox}"
    response = requests.get(url)
    
    if response.status_code == 200:
        print(f"Number of Incidents: {len(response.json()['incidents'])}") # analysis purpose
        return response.json()
    else:
        return {"error": f"Failed to fetch data. Status code: {response.status_code}"}
    
    return response

### Helper functions

In [5]:
def save_data_to_json(filename, data):
    """
    save data to a JSON file
    """
    with open(filename, "w") as json_file:
        json.dump(data, json_file, indent=4)

In [6]:
def json_to_df(filename):
    """
    save data to a dataframe
    """
    # Load the JSON data
    with open(filename, 'r') as f:
        data = json.load(f)
    # Normalize the JSON data to a flat table (DataFrame)
    incidents_df = pd.json_normalize(data['incidents'])
    # Replace the 'properties.iconCategory' with the corresponding descriptive labels
    incidents_df['properties.iconCategory'] = incidents_df['properties.iconCategory'].map(ICON_MAPPING)

    return incidents_df

## 1-1.

### Visualization with `Folium`

In [7]:
api_key = "N8xhZrbfY7NZEqbqIeEedz4xUBmXARBG"
bbox = "-74.25,40.50,-73.70,40.95"  # Bounding box for NYC

traffic_incidents_data = get_traffic_incidents(api_key, bbox)

if "error" not in traffic_incidents_data:
    filename = "traffic_incidents_data.json"
    # Save data to a JSON file
    save_data_to_json(filename, traffic_incidents_data)
    # Create a dataframe from the JSON file
    incidents_df = json_to_df(filename)
    print(f"Succeeded: {incidents_df.shape[0]} traffic incidents in data frame!")
else:
    print(traffic_incidents_data["error"])

Number of Incidents: 1902
Succeeded: 1902 traffic incidents in data frame!


In [8]:
incidents_df.head()

,type,properties.iconCategory,geometry.type,geometry.coordinates
0,Feature,Jam,LineString,"[[-74.2489609714, 40.8405697552], [-74.2485961..."
1,Feature,Jam,LineString,"[[-74.2498635348, 40.8335598447], [-74.2496556..."
2,Feature,Jam,LineString,"[[-74.2434731718, 40.6357603154], [-74.2449577..."
3,Feature,Jam,LineString,"[[-74.2434235509, 40.8403485111], [-74.2453963..."
4,Feature,Jam,LineString,"[[-74.2452930506, 40.7008119257], [-74.2453145..."


In [9]:
# Access the coordinates of the first row
coordinates_first_row = incidents_df.loc[0, 'geometry.coordinates']
# Display the coordinates
print(coordinates_first_row)

[[-74.2489609714, 40.8405697552], [-74.248596191, 40.8407199776], [-74.2484325762, 40.8408218801], [-74.2482260462, 40.8412523797], [-74.2482045885, 40.8413838309], [-74.2482287284, 40.8414870639], [-74.2482984658, 40.8416211779], [-74.2483842965, 40.841693593]]


In [10]:
incident_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

# Create separate feature groups for each incident type
incident_types = incidents_df['properties.iconCategory'].unique()

# Dictionary to store feature groups by incident type
feature_groups = {incident_type: folium.FeatureGroup(name=incident_type) for incident_type in incident_types}

# Add incidents to the respective feature group with custom icons
for index, row in incidents_df.iterrows():
    coordinates = row['geometry.coordinates'][0]  # Extract coordinates
    incident_type = row['properties.iconCategory']
    
    # Get the icon and color for the incident type
    icon_info = ICON_STYLES.get(incident_type, {'icon': 'info-sign', 'color': 'green'})  # Default icon if not found
    
    # Add markers with custom icons to the corresponding feature group
    folium.Marker(
        location=[coordinates[1], coordinates[0]],  # Coordinates in [lat, lon]
        popup=f"Incident: {incident_type}",
        icon=folium.Icon(icon=icon_info['icon'], color=icon_info['color'])
    ).add_to(feature_groups[incident_type])

# Add all feature groups to the map
for fg in feature_groups.values():
    fg.add_to(incident_map)

# Add layer control to toggle the feature groups
folium.LayerControl().add_to(incident_map)

# Save the map to an HTML file
incident_map.save('traffic_incidents_map_static.html')

## 2-2.

### Real-time Visualization with `Folium`

In [11]:
def fetch_and_update_map():
    """
    make API requests and create and update map
    """
    #### From section "Main"
    traffic_incidents_data = get_traffic_incidents(api_key, bbox)

    if "error" not in traffic_incidents_data:
        filename = "traffic_incidents_data.json"
        # Save data to a JSON file
        save_data_to_json(filename, traffic_incidents_data)
        # Create a dataframe from the JSON file
        incidents_df = json_to_df(filename)
        print(f"Succeeded: {incidents_df.shape[0]} traffic incidents in data frame!")
    else:
        print(traffic_incidents_data["error"])
    #### 
    
    # Initialize a new map
    incident_map = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

    # Create separate feature groups for each incident type
    incident_types = incidents_df['properties.iconCategory'].unique()
    feature_groups = {incident_type: folium.FeatureGroup(name=incident_type) for incident_type in incident_types}

    # Add incidents to the respective feature group with custom icons
    for index, row in incidents_df.iterrows():
        coordinates = row['geometry.coordinates'][0]  # Extract coordinates
        incident_type = row['properties.iconCategory']
        
        # Get the icon and color for the incident type
        icon_info = ICON_STYLES.get(incident_type, {'icon': 'info-sign', 'color': 'green'})  # Default icon
        
        # Add markers with custom icons to the corresponding feature group
        folium.Marker(
            location=[coordinates[1], coordinates[0]],  # Coordinates in [lat, lon]
            popup=f"Incident: {incident_type}",
            icon=folium.Icon(icon=icon_info['icon'], color=icon_info['color'])
        ).add_to(feature_groups[incident_type])

    # Add all feature groups to the map
    for fg in feature_groups.values():
        fg.add_to(incident_map)

    # Add layer control to toggle the feature groups
    folium.LayerControl().add_to(incident_map)

    # Save the map to an HTML file
    incident_map.save('traffic_incidents_map.html')
    print("Map updated with latest incidents\n")

In [12]:
api_key = "N8xhZrbfY7NZEqbqIeEedz4xUBmXARBG"
bbox = "-74.25,40.50,-73.70,40.95"  # Bounding box for NYC

# Schedule the task to run every 2 minutes
schedule.every(2).minutes.do(fetch_and_update_map)

# Keep the script running to execute the task
while True:
    schedule.run_pending()
    time.sleep(1)

Number of Incidents: 1993
Succeeded: 1993 traffic incidents in data frame!
Map updated with latest incidents

Number of Incidents: 2035
Succeeded: 2035 traffic incidents in data frame!
Map updated with latest incidents

Number of Incidents: 2027
Succeeded: 2027 traffic incidents in data frame!
Map updated with latest incidents



KeyboardInterrupt: 

**Report:** 


1. (every 2 minutes) 30 requests/hour $\times$ 24 hours = 720 requests **: current code**
2. (every 10 minutes) 6 requests/hour $\times$ 24 hours = 144 requests
3. (every 60 minutes) 1 request/hour $\times$ 24 hours = 24 requests

# 2.

### With customized fields

In [ ]:
# With Field
def get_traffic_incidents_customized(api_key, bbox, 
                         baseURL = "api.tomtom.com", 
                         versionNumber = 5,
                         fields="{incidents{type,geometry{type,coordinates},properties{iconCategory,magnitudeOfDelay,events{description,code,iconCategory},startTime,endTime,from,to,length,delay,roadNumbers,timeValidity,probabilityOfOccurrence,numberOfReports}}}"):
    """
    for real-time data on accidents, congestion, road closures, etc.
    """
    # [GUIDE] https://developer.tomtom.com/traffic-api/documentation/traffic-incidents/incident-details
    url = f"https://{baseURL}/traffic/services/{versionNumber}/incidentDetails?key={api_key}&bbox={bbox}&fields={fields}"
    response = requests.get(url)
    
    if response.status_code == 200:
        print(f"Number of Incidents: {len(response.json()['incidents'])}") # analysis purpose
        return response.json()
    else:
        return {"error": f"Failed to fetch data. Status code: {response.status_code}"}

### Main

In [ ]:
api_key = "N8xhZrbfY7NZEqbqIeEedz4xUBmXARBG"
bbox = "-74.25,40.50,-73.70,40.95"  # Bounding box for NYC

traffic_incidents_data = get_traffic_incidents_customized(api_key, bbox)

if "error" not in traffic_incidents_data:
    filename = "traffic_incidents_data_customized.json"
    # Save data to a JSON file
    save_data_to_json(filename, traffic_incidents_data)
    # Create a dataframe from the JSON file
    df = json_to_df(filename)
    print(f"Succeeded: {df.shape[0]} traffic incidents in data frame!")
else:
    print(traffic_incidents_data["error"])

In [ ]:
df.head()

In [ ]:
# Access the coordinates of the first row
coordinates_first_row = df.loc[0, 'geometry.coordinates']
# Display the coordinates
print(coordinates_first_row)